In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

print("🐺 WALTZ INVESTIGATION")
print("="*70)
print("Full effort. No promises. Just looking at the data.")
print("="*70)

In [ ]:
# Our universe - organized by sector
SECTORS = {
    'Quantum': ['IONQ', 'RGTI', 'QBTS', 'QUBT', 'ARQQ'],
    'Nuclear': ['UUUU', 'USAR', 'NXE', 'DNN', 'LEU'],
    'Space': ['RKLB', 'ASTS', 'LUNR', 'PL', 'SIDU'],
    'Semi': ['NVTS', 'BRSH', 'NXPI', 'SWKS', 'MRVL'],
    'AI_Infra': ['AVGO', 'ARM', 'NVDA', 'SMCI', 'PLTR']
}

ALL_TICKERS = [t for sector in SECTORS.values() for t in sector]
print(f"Universe: {len(ALL_TICKERS)} tickers across {len(SECTORS)} sectors")
for sector, tickers in SECTORS.items():
    print(f"  {sector}: {', '.join(tickers)}")

In [ ]:
# Pull 6 weeks of data - Dec 1 to Jan 7
start_date = '2025-12-01'
end_date = '2026-01-07'

all_data = {}
failed = []

print(f"\nPulling data from {start_date} to {end_date}...")
print("-"*70)

for ticker in ALL_TICKERS:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        
        if not hist.empty and len(hist) > 5:
            all_data[ticker] = hist
            print(f"✓ {ticker}: {len(hist)} days")
        else:
            failed.append(ticker)
            print(f"✗ {ticker}: No data")
    except Exception as e:
        failed.append(ticker)
        print(f"✗ {ticker}: {e}")

print(f"\nLoaded: {len(all_data)} tickers")
if failed:
    print(f"Failed: {failed}")

## STEP 1: What ran in the last 6 weeks?
Simple ranking - who moved the most?

In [ ]:
# Calculate total return for each ticker over the period
returns = {}

for ticker, df in all_data.items():
    if len(df) > 0:
        start_price = df['Close'].iloc[0]
        end_price = df['Close'].iloc[-1]
        total_return = ((end_price - start_price) / start_price) * 100
        returns[ticker] = total_return

# Sort by return
sorted_returns = sorted(returns.items(), key=lambda x: x[1], reverse=True)

print("\n" + "="*70)
print("TOTAL RETURNS: Dec 1 - Jan 7")
print("="*70)
print("\nWINNERS (Top 10):")
for ticker, ret in sorted_returns[:10]:
    # Find sector
    sector = [s for s, tickers in SECTORS.items() if ticker in tickers][0]
    print(f"  {ticker:6} ({sector:8}): {ret:+.1f}%")

print("\nLOSERS (Bottom 5):")
for ticker, ret in sorted_returns[-5:]:
    sector = [s for s, tickers in SECTORS.items() if ticker in tickers][0]
    print(f"  {ticker:6} ({sector:8}): {ret:+.1f}%")

## STEP 2: Sector Performance
Did whole sectors move together? Or individual tickers?

In [ ]:
print("\n" + "="*70)
print("SECTOR PERFORMANCE: Dec 1 - Jan 7")
print("="*70)

sector_returns = {}

for sector, tickers in SECTORS.items():
    sector_rets = []
    print(f"\n{sector}:")
    print("-"*50)
    
    for ticker in tickers:
        if ticker in returns:
            ret = returns[ticker]
            sector_rets.append(ret)
            print(f"  {ticker:6}: {ret:+.1f}%")
    
    if sector_rets:
        avg = np.mean(sector_rets)
        sector_returns[sector] = avg
        print(f"  ------")
        print(f"  AVG:    {avg:+.1f}%")

print("\n" + "="*70)
print("SECTOR RANKING:")
print("="*70)
for sector, ret in sorted(sector_returns.items(), key=lambda x: x[1], reverse=True):
    print(f"  {sector:10}: {ret:+.1f}%")

## STEP 3: TIMING - When did each ticker start running?
Find the DATE when each ticker's run began (first day of 3+ consecutive up days)

In [ ]:
def find_run_starts(df, min_gain=10, window=5):
    """
    Find dates where a significant run started.
    A 'run start' is when price gains 10%+ over next 5 days.
    """
    runs = []
    
    for i in range(len(df) - window):
        start_price = df['Close'].iloc[i]
        
        # Check if any of the next 5 days hit 10%+ gain
        for j in range(1, window + 1):
            if i + j < len(df):
                future_price = df['Close'].iloc[i + j]
                gain = ((future_price - start_price) / start_price) * 100
                
                if gain >= min_gain:
                    runs.append({
                        'start_date': df.index[i],
                        'peak_date': df.index[i + j],
                        'gain': gain,
                        'days': j
                    })
                    break
    
    return runs

print("\n" + "="*70)
print("RUN START DATES - When did momentum begin?")
print("="*70)

all_runs = []

for ticker, df in all_data.items():
    runs = find_run_starts(df, min_gain=15, window=7)
    
    if runs:
        sector = [s for s, tickers in SECTORS.items() if ticker in tickers][0]
        for run in runs[:2]:  # Top 2 runs per ticker
            start = run['start_date'].strftime('%Y-%m-%d')
            all_runs.append({
                'ticker': ticker,
                'sector': sector,
                'start': start,
                'gain': run['gain'],
                'days': run['days']
            })

# Sort by start date
all_runs = sorted(all_runs, key=lambda x: x['start'])

print("\nRuns detected (sorted by start date):")
print("-"*70)
for run in all_runs:
    print(f"  {run['start']} | {run['ticker']:6} ({run['sector']:8}) | +{run['gain']:.0f}% in {run['days']}d")

## STEP 4: THEIR CALENDAR - Week by Week Analysis
What happened each week? What started? What stopped?

In [ ]:
# Calculate weekly returns for each ticker
print("\n" + "="*70)
print("WEEK BY WEEK - What moved when?")
print("="*70)

# Define weeks
weeks = [
    ('Dec 1-5', '2025-12-01', '2025-12-05'),
    ('Dec 8-12', '2025-12-08', '2025-12-12'),
    ('Dec 15-19', '2025-12-15', '2025-12-19'),
    ('Dec 22-26', '2025-12-22', '2025-12-26'),
    ('Dec 29-Jan 2', '2025-12-29', '2026-01-02'),
    ('Jan 5-7', '2026-01-05', '2026-01-07')
]

for week_name, start, end in weeks:
    print(f"\n{week_name}:")
    print("-"*50)
    
    week_returns = []
    
    for ticker, df in all_data.items():
        # Filter to week
        mask = (df.index >= start) & (df.index <= end)
        week_df = df[mask]
        
        if len(week_df) >= 2:
            ret = ((week_df['Close'].iloc[-1] - week_df['Close'].iloc[0]) / week_df['Close'].iloc[0]) * 100
            sector = [s for s, tickers in SECTORS.items() if ticker in tickers][0]
            week_returns.append((ticker, sector, ret))
    
    # Sort and show top 5 movers
    week_returns = sorted(week_returns, key=lambda x: x[2], reverse=True)
    
    print("  Top movers:")
    for ticker, sector, ret in week_returns[:5]:
        if ret > 0:
            print(f"    {ticker:6} ({sector:8}): {ret:+.1f}%")
    
    # Show sector averages for the week
    print("  Sector averages:")
    for sector in SECTORS.keys():
        sector_rets = [r[2] for r in week_returns if r[1] == sector]
        if sector_rets:
            avg = np.mean(sector_rets)
            if abs(avg) > 2:
                print(f"    {sector:10}: {avg:+.1f}%")

## STEP 5: ROTATION PATTERNS - Did sectors hand off to each other?

In [ ]:
# Build daily sector returns
print("\n" + "="*70)
print("SECTOR ROTATION - Daily sector returns")
print("="*70)

# Get common dates
common_dates = None
for ticker, df in all_data.items():
    if common_dates is None:
        common_dates = set(df.index)
    else:
        common_dates = common_dates.intersection(set(df.index))

common_dates = sorted(list(common_dates))

# Build sector daily returns dataframe
sector_daily = pd.DataFrame(index=common_dates)

for sector, tickers in SECTORS.items():
    sector_returns = []
    
    for date in common_dates:
        daily_rets = []
        for ticker in tickers:
            if ticker in all_data:
                df = all_data[ticker]
                if date in df.index:
                    idx = df.index.get_loc(date)
                    if idx > 0:
                        prev = df['Close'].iloc[idx-1]
                        curr = df['Close'].iloc[idx]
                        daily_rets.append(((curr - prev) / prev) * 100)
        
        sector_returns.append(np.mean(daily_rets) if daily_rets else 0)
    
    sector_daily[sector] = sector_returns

# Show last 10 days
print("\nLast 10 trading days - Sector daily returns:")
print("-"*70)
print(sector_daily.tail(10).round(1).to_string())

In [ ]:
# Find rotation: When one sector peaks, does another start?
print("\n" + "="*70)
print("ROTATION DETECTION - Handoffs between sectors")
print("="*70)

# Calculate 3-day rolling returns for each sector
for sector in SECTORS.keys():
    sector_daily[f'{sector}_3d'] = sector_daily[sector].rolling(3).sum()

# Find days where one sector peaked and another started
print("\nLooking for rotation patterns:")
print("(When one sector's 3-day return peaks, does another start?)")
print("-"*70)

for i in range(5, len(sector_daily) - 3):
    date = sector_daily.index[i]
    date_str = date.strftime('%Y-%m-%d')
    
    # Check each sector
    for sector in SECTORS.keys():
        col = f'{sector}_3d'
        
        # Is this a local peak? (Higher than prev 2 days and next 2 days)
        current = sector_daily[col].iloc[i]
        prev_avg = sector_daily[col].iloc[i-2:i].mean()
        next_avg = sector_daily[col].iloc[i+1:i+3].mean()
        
        if current > 10 and current > prev_avg and current > next_avg + 5:
            # This sector peaked - what started?
            print(f"\n{date_str}: {sector} peaked ({current:.1f}% over 3d)")
            
            # Check other sectors' next 3 days
            for other in SECTORS.keys():
                if other != sector:
                    other_col = f'{other}_3d'
                    next_3d = sector_daily[other_col].iloc[i+1:i+4]
                    if len(next_3d) > 0:
                        max_next = next_3d.max()
                        if max_next > 8:
                            print(f"  → {other} ran after ({max_next:.1f}% over next 3d)")

## STEP 6: Jan 6 Winners Deep Dive
SIDU, USAR, NVTS, ASTS - What did the week BEFORE look like?

In [ ]:
JAN6_WINNERS = ['SIDU', 'USAR', 'NVTS', 'ASTS']

print("\n" + "="*70)
print("JAN 6 WINNERS - The Week Before")
print("="*70)

for ticker in JAN6_WINNERS:
    if ticker not in all_data:
        print(f"\n{ticker}: No data")
        continue
    
    df = all_data[ticker]
    sector = [s for s, tickers in SECTORS.items() if ticker in tickers][0]
    
    print(f"\n{ticker} ({sector}):")
    print("-"*70)
    
    # Get Dec 29 - Jan 6
    mask = df.index >= '2025-12-29'
    recent = df[mask]
    
    for date, row in recent.iterrows():
        date_str = date.strftime('%Y-%m-%d')
        daily_ret = ((row['Close'] - row['Open']) / row['Open']) * 100
        
        # Volume analysis
        vol_avg = df['Volume'].rolling(20).mean().loc[date] if date in df.index else row['Volume']
        vol_ratio = row['Volume'] / vol_avg if vol_avg > 0 else 1
        
        # Price position in range
        range_pos = (row['Close'] - row['Low']) / (row['High'] - row['Low']) * 100 if row['High'] != row['Low'] else 50
        
        print(f"  {date_str}: ${row['Close']:7.2f} | {daily_ret:+5.1f}% | Vol {vol_ratio:.1f}x | Close at {range_pos:.0f}% of range")

## STEP 7: What's HOT Right Now?
Current state of each sector and ticker

In [ ]:
print("\n" + "="*70)
print("CURRENT STATE - What's hot? What's cold?")
print("="*70)

current_state = []

for ticker, df in all_data.items():
    if len(df) < 10:
        continue
    
    sector = [s for s, tickers in SECTORS.items() if ticker in tickers][0]
    
    # Last close
    last_close = df['Close'].iloc[-1]
    
    # 5-day return
    ret_5d = ((df['Close'].iloc[-1] - df['Close'].iloc[-5]) / df['Close'].iloc[-5]) * 100
    
    # RSI approximation (simplified)
    changes = df['Close'].diff().iloc[-14:]
    gains = changes[changes > 0].sum()
    losses = abs(changes[changes < 0].sum())
    if losses > 0:
        rs = gains / losses
        rsi = 100 - (100 / (1 + rs))
    else:
        rsi = 100
    
    # Volume trend (last 3 days vs 20-day avg)
    vol_3d = df['Volume'].iloc[-3:].mean()
    vol_20d = df['Volume'].iloc[-20:].mean()
    vol_trend = vol_3d / vol_20d if vol_20d > 0 else 1
    
    current_state.append({
        'ticker': ticker,
        'sector': sector,
        'price': last_close,
        '5d_ret': ret_5d,
        'rsi': rsi,
        'vol_trend': vol_trend
    })

# Sort by 5d return
current_state = sorted(current_state, key=lambda x: x['5d_ret'], reverse=True)

print("\nHOT (High RSI, High Momentum):")
for item in current_state[:8]:
    status = '🔥' if item['rsi'] > 65 else '  '
    print(f"{status} {item['ticker']:6} ({item['sector']:8}): {item['5d_ret']:+6.1f}% (5d) | RSI {item['rsi']:.0f} | Vol {item['vol_trend']:.1f}x")

print("\nCOLD (Low RSI, Beaten Down):")
for item in current_state[-5:]:
    status = '❄️' if item['rsi'] < 35 else '  '
    print(f"{status} {item['ticker']:6} ({item['sector']:8}): {item['5d_ret']:+6.1f}% (5d) | RSI {item['rsi']:.0f} | Vol {item['vol_trend']:.1f}x")

## WHAT WE SEE (No Conclusions Yet)

This notebook shows us THE DATA. Not theories.

We can see:
1. Which sectors performed over 6 weeks
2. When runs started (specific dates)
3. Week-by-week rotation
4. What the Jan 6 winners looked like the week before
5. Current state - what's hot, what's cold

We can't see (yet):
- WHY things moved (need catalyst data)
- Insider activity (need Form 4 checks)
- Options flow (need different data source)

**Next step:** Look at this data. See what patterns emerge. Don't force theories.